실습:삼성전자 주가 데이터를 분석하는 어플리케이션을 function(tool)으로 수정하여,다른 종목의 데이터를 정해지 내용대로(종목,시작날짜,종료날짜) 가져와서 답하는 어플리케이션을 작성하시오.

In [1]:
import os
import json
from openai import OpenAI

client = OpenAI()
client.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:

import openai
import json

openai.api_key = os.getenv("OPENAI_API_KEY")

# 🔧 1. Function 정의 (여러 개 등록)
functions = [
    {
        "name": "get_weather",
        "description": "도시의 현재 날씨를 조회합니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {"type": "string", "description": "도시 이름"}
            },
            "required": ["symbol"]
        }
    },
    {
        "name": "get_stock_price",
        "description": "주식 심볼의 현재 가격을 조회합니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "symbol": {"type": "string", "description": "주식 심볼 (예: AAPL, 035420.KQ)"}
            },
            "required": ["symbol"]
        }
   },
    {
        "name": "search_news",
        "description": "특정 키워드로 최신 뉴스를 검색합니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {"type": "string", "description": "검색 키워드"}
            },
            "required": ["query"]
        }
    }
]

messages = [
    {"role": "user", "content": "서울 날씨 알려줘"}  # GPT가 어떤 함수를 쓸지 추론함
]

response = openai.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    functions=functions,
    function_call="auto"  
)


if response.choices[0].finish_reason == "function_call":
    call = response.choices[0].message.function_call
    func_name = call.name
    arguments = json.loads(call.arguments)


    def get_weather(location):
        return {"location": location, "temp": "28도", "condition": "맑음"}

    def get_stock_price(symbol):
        return {"symbol": symbol, "price": "207,500원"}

    def search_news(query):
        return {"query": query, "headlines": ["뉴스1", "뉴스2", "뉴스3"]}

    # 함수 이름에 따라 실제 실행
    result = None
    if func_name == "get_weather":
        result = get_weather(**arguments)
    elif func_name == "get_stock_price":
        result = get_stock_price(**arguments)
    elif func_name == "search_news":
        result = search_news(**arguments)


    followup = openai.chat.completions.create(
        model="gpt-4",
        messages=[
            *messages,
            response.choices[0].message,
            {"role": "function", "name": func_name, "content": json.dumps(result)}
        ]
    )

    print(f"\nGPT 응답:\n{followup.choices[0].message.content}")

else:
    print("함수 호출 없이 GPT 응답:\n", response.choices[0].message.content)




🤖 GPT 응답:
현재 서울의 기온은 28도이고 날씨는 맑음입니다.
